In [1]:
import torch

import torch.nn.functional as F

In [42]:
latent_dim = 5
categorical_dim = 2  # one-of-K vector

def sample_gumbel(shape, eps=1e-10):
    U = torch.rand(shape)
    return -torch.log(-torch.log(U + eps) + eps)

def gumbel_softmax_sample(logits, temperature):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)

def gumbel_softmax(logits, temperature, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)

    if not hard:
        return y#.view(-1, latent_dim * categorical_dim)

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y

    return y_hard#.view(-1, latent_dim * categorical_dim)

In [43]:
example = torch.rand(latent_dim, categorical_dim)
#example

In [50]:
torch.set_printoptions(threshold=5)
y = gumbel_softmax(example, 1, hard=False)
y

tensor([[0.1174, 0.8826],
        [0.3776, 0.6224],
        [0.7146, 0.2854],
        [0.4186, 0.5814],
        [0.6976, 0.3024]])

In [47]:
F.gumbel_softmax(example, tau=1, hard=False, eps=1e-10, dim=-1)

tensor([[0.8440, 0.1560],
        [0.6549, 0.3451],
        [0.3913, 0.6087],
        [0.2252, 0.7748],
        [0.0544, 0.9456]])

In [41]:
shape = y.size()
_, ind = y.max(dim=-1)
y_hard = torch.zeros_like(y).view(-1, shape[-1])
y_hard.scatter_(1, ind.view(-1, 1), 1)
#print(y_hard.size())
#print(y_hard)
#y_hard = (y_hard - y).detach() + y
y_hard

torch.Size([5, 2])
tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])


tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])